In [52]:
import mysql.connector

db = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "root",
    database = "CSCI_630_GA2"
    )

mycursor = db.cursor()


In [53]:
import pandas as pd

for i in range(1,10):

    file = f"Table1/table-1-{i}.xlsx"

    df = pd.read_excel(file, header = 5)

    df.iloc[:, 0] = df.iloc[:, 0].replace("\t", "", regex=True) 

    df = df.drop([0, 1, 3, 15, 16, 18, 30, 31, 33])

    df.replace('Z', 0, inplace=True)

    df.set_index(df.columns[0], inplace=True)

    both_sex = df.iloc[:12].copy()

    male = df.iloc[12:24].copy()

    female = df.iloc[24:36].copy()

    genders = {
    'both': both_sex,
    'male': male,
    'female': female
    }
    
    for gender_type, gender_data in genders.items():
        create_table_query = f"CREATE TABLE IF NOT EXISTS table_1_{i}_{gender_type} (`Age groups` VARCHAR(100), "
        count = 0
        for j in gender_data.columns:
            if count > 0:
                create_table_query += ", "
            else:
                count += 1
            create_table_query += "`" + j + "`" + " int UNSIGNED"

        create_table_query += ", PRIMARY KEY (`Age groups`))"
        mycursor.execute(create_table_query)
        db.commit()

        batch_size = 100;
        batch_i = 0
        
        vallist = []
        #placeholder = []
        for k in range(len(gender_data)):
            batch_i = batch_i + 1
            values = gender_data.iloc[k].tolist()
            values.insert(0, gender_data.index[k])
            vallist.append(values)
            placeholders = ", ".join(['%s'] * len(values))
            query = (f"INSERT INTO table_1_{i}_{gender_type} VALUES ({placeholders})")
            if(batch_i % batch_size == 0):
                print(vallist)
                mycursor.executemany(query, vallist)
                db.commit()
        
        print(vallist)
        print(query)
        mycursor.executemany(query, vallist)
        db.commit()
            


[['  18 to 24 years', 28981.0, 43.0, 42.0, 80.0, 200.0, 203.0, 459.0, 3432.0, 10065.0, 9113.0, 634.0, 1080.0, 3305.0, 279.0, 34.0, 11.0], ['  25 to 29 years', 21747.0, 14.0, 36.0, 82.0, 188.0, 149.0, 166.0, 546.0, 6337.0, 3551.0, 815.0, 1252.0, 6390.0, 1804.0, 200.0, 217.0], ['  30 to 34 years', 22836.0, 25.0, 58.0, 142.0, 187.0, 217.0, 247.0, 613.0, 5767.0, 3409.0, 922.0, 1544.0, 6344.0, 2567.0, 292.0, 504.0], ['  35 to 39 years', 21992.0, 73.0, 85.0, 164.0, 244.0, 240.0, 274.0, 640.0, 5426.0, 3076.0, 911.0, 1369.0, 5786.0, 2762.0, 400.0, 543.0], ['  40 to 44 years', 21198.0, 47.0, 124.0, 314.0, 282.0, 321.0, 280.0, 548.0, 5283.0, 2784.0, 916.0, 1219.0, 5395.0, 2734.0, 424.0, 524.0], ['  45 to 49 years', 19430.0, 48.0, 107.0, 305.0, 280.0, 273.0, 243.0, 476.0, 4927.0, 2679.0, 966.0, 1115.0, 4713.0, 2539.0, 331.0, 427.0], ['  50 to 54 years', 20700.0, 54.0, 133.0, 297.0, 299.0, 297.0, 227.0, 603.0, 5895.0, 2618.0, 995.0, 1196.0, 4979.0, 2377.0, 264.0, 467.0], ['  55 to 59 years', 21004

In [54]:
mycursor.execute("SELECT * from table_1_1_both")
result = mycursor.fetchall()
for i in range(len(result)):
    print(result[i])

('  18 to 24 years', 28981, 43, 42, 80, 200, 203, 459, 3432, 10065, 9113, 634, 1080, 3305, 279, 34, 11)
('  25 to 29 years', 21747, 14, 36, 82, 188, 149, 166, 546, 6337, 3551, 815, 1252, 6390, 1804, 200, 217)
('  30 to 34 years', 22836, 25, 58, 142, 187, 217, 247, 613, 5767, 3409, 922, 1544, 6344, 2567, 292, 504)
('  35 to 39 years', 21992, 73, 85, 164, 244, 240, 274, 640, 5426, 3076, 911, 1369, 5786, 2762, 400, 543)
('  40 to 44 years', 21198, 47, 124, 314, 282, 321, 280, 548, 5283, 2784, 916, 1219, 5395, 2734, 424, 524)
('  45 to 49 years', 19430, 48, 107, 305, 280, 273, 243, 476, 4927, 2679, 966, 1115, 4713, 2539, 331, 427)
('  50 to 54 years', 20700, 54, 133, 297, 299, 297, 227, 603, 5895, 2618, 995, 1196, 4979, 2377, 264, 467)
('  55 to 59 years', 21004, 111, 168, 248, 258, 243, 299, 609, 6462, 2947, 1079, 1471, 4575, 1840, 257, 436)
('  60 to 64 years', 21174, 74, 150, 277, 260, 223, 318, 689, 6791, 3183, 1026, 1318, 4324, 1840, 326, 374)
('  65 to 69 years', 18533, 73, 134, 253,

In [55]:
import pandas as pd

file = "Table3/table-3.xlsx"

df = pd.read_excel(file, header = [3, 4])

df = df.replace("\t", "", regex=True)

df = df.replace('Z', 0)

df = df.drop([1, 10, 19, 21, 27, 33, 39])

df = df.iloc[:37]

df = df.set_index(df.columns[0])

# print(df)

number_columns = [col for i, col in enumerate(df.columns) if i % 2 == 0]

df.drop(columns=number_columns, inplace=True)

create_table_query = f"CREATE TABLE IF NOT EXISTS table_3 (Degree VARCHAR(100), "
count = 0
for i in df.columns:
    print(i)
    if count > 0:
        create_table_query += ", "
    else:
        count += 1
    create_table_query += "`" + i[0] + "`" + " int UNSIGNED"

# print(create_table_query)
create_table_query += ", PRIMARY KEY (degree))"
mycursor.execute(create_table_query)
db.commit()

batch_size = 100;
batch_i = 0
        
vallist = []
#placeholder = []
for k in range(len(df)):
    batch_i = batch_i + 1
    values = df.iloc[k].tolist()
    values.insert(0, df.index[k])
    vallist.append(values)
    placeholders = ", ".join(['%s'] * len(values))
    query = (f"INSERT INTO table_3 VALUES ({placeholders})")
    if(batch_i % batch_size == 0):
        print(vallist)
        mycursor.executemany(query, vallist)
        db.commit()
 
print(vallist)
mycursor.executemany(query, vallist)
db.commit()



('All people', 'Percent')
('Male', 'Percent')
('Female', 'Percent')
('25 to 34 years old', 'Percent')
('35 to 54 years old', 'Percent')
('55 years and older', 'Percent')
('White', 'Percent')
('Non-Hispanic White', 'Percent')
('Black', 'Percent')
('Asian', 'Percent')
('Hispanic (of any race)', 'Percent')
[['Total', 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], [' Less than 1 year, no diploma', 0.3, 0.3, 0.3, 0.1, 0.3, 0.5, 0.3, 0.1, 0.3, 0.9, 1.2], [' 1st-4th grade, no diploma', 0.7, 0.7, 0.6, 0.2, 0.5, 0.9, 0.7, 0.1, 0.4, 0.7, 3.1], [' 5th-6th grade, no diploma', 1.2, 1.2, 1.2, 0.5, 1.3, 1.5, 1.3, 0.2, 0.6, 1.4, 5.8], [' 7th-8th grade, no diploma', 1.3, 1.4, 1.3, 0.8, 1.3, 1.5, 1.4, 0.7, 1.0, 1.4, 3.8], [' 9th grade, no diploma', 1.2, 1.3, 1.1, 0.8, 1.4, 1.2, 1.2, 0.6, 1.0, 0.8, 3.9], [' 10th grade, no diploma', 1.3, 1.4, 1.2, 0.9, 1.2, 1.5, 1.3, 1.0, 1.5, 0.8, 2.2], [' 11th grade, no diploma', 1.6, 1.7, 1.5, 1.3, 1.5, 1.8, 1.4, 1.2, 3.2, 0.4, 2.3], [' 1

In [56]:
mycursor.execute("SELECT * from table_3")
result = mycursor.fetchall()
for i in range(len(result)):
    print(result[i])

(' 10th grade, GED', 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0)
(' 10th grade, no diploma', 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2)
(' 11th grade, GED', 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1)
(' 11th grade, no diploma', 2, 2, 2, 1, 2, 2, 1, 1, 3, 0, 2)
(' 12th grade, GED', 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1)
(' 12th grade, no diploma', 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3)
(' 1st-4th grade, GED', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(' 1st-4th grade, no diploma', 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 3)
(' 5th-6th grade, GED', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(' 5th-6th grade, no diploma', 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 6)
(' 7th-8th grade, GED', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(' 7th-8th grade, no diploma', 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 4)
(' 9th grade, GED', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(' 9th grade, no diploma', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4)
(" Bachelor's degree", 23, 23, 24, 29, 25, 20, 24, 26, 17, 33, 15)
(' Doctorate degree', 2, 2, 2, 2, 2, 2, 2, 2, 1, 5, 1)
(" Four or more years of college, academic/associate's", 